## Data Cleaning

### Tidying Geolocations
#### Removing data points (lat long) that fall outside City of Melbourne

In [52]:
import sys
# Install packages
# !{sys.executable} -m pip install shapely
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install geopandas
# !{sys.executable} -m pip install geojson

# Load packages
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import numpy as np
import pandas as pd
import geopandas as gpd
import geojson
import shapely.wkt
import datetime


home = "/Users/declanwatson/Documents/Info_vis/info_vis_ass_3/"


In [73]:
# Create City of Melbourne Polygon from geo json object
path = home + "data/Transport/PTV_combined.csv"
df = pd.read_csv(path)


with open(home + "data/Municipal boundary.geojson") as f:
    gj = geojson.load(f)
points = gj['features'][0]['geometry']['coordinates'][0][0]
polygon = Polygon(points)

In [68]:
#Check if coordinate in city of melbourne and return modified dataframe
lats = df['LATITUDE'].to_list()
longs = df['LONGITUDE'].to_list()

point_list = [Point([longs[i], lats[i]]) for i in range(len(lats))]
in_com = [polygon.contains(P) for P in point_list]
df['In City Of Melbourne'] = in_com
df = df[df['In City Of Melbourne']== True]

In [75]:
# Save df
# df.to_csv(home + "data/ptv_combined_city_of_melbourne.csv", index=False)

### Averaging Temperature for sensor locations

In [87]:
# Average by month temperature
path = home + 'data/Environment/Microclimate_Sensor_Readings.csv'
df = pd.read_csv(path)

ugly_date = df['local_time'].tolist()
clean_date = [datetime.datetime.strptime(temp_date, "%d/%m/%Y %H:%M").month for temp_date in ugly_date]
df['month'] = clean_date

In [89]:
df

,id,site_id,gateway_hub_id,sensor_id,value,local_time,type,units,longitude,latitude,month
0,1,1001,arc1045,0a.EPA-1h,3.90,15/11/2019 9:00,PM2.5-EPA-1h,ug/m^3,144.966498,-37.800780,11
1,2,1001,arc1045,0a.EPA-1h.NOPK,3.90,15/11/2019 9:00,PM2.5-EPA-1h-NOPK,ug/m^3,144.966498,-37.800780,11
2,3,1001,arc1045,0a.EPA-1h.NOPK.EPA-24h,5.50,15/11/2019 9:00,PM2.5-EPA-1h-NOPK-EPA-24h,ug/m^3,144.966498,-37.800780,11
3,4,1001,arc1045,0a.EPA-1h.PKIND,1.00,15/11/2019 9:00,PM2.5-EPA-1h-PKIND,0/1,144.966498,-37.800780,11
4,5,1001,arc1045,0a.EPA-24h,5.50,15/11/2019 9:00,PM2.5-EPA-24h,ug/m^3,144.966498,-37.800780,11
...,...,...,...,...,...,...,...,...,...,...,...
1048570,1542104,1005,arc1050,0b.EPA-1h.PKIND,1.00,8/8/2020 3:00,PM10-EPA-1h-PKIND,0/1,144.965058,-37.800616,8
1048571,1542105,1001,arc1045,5b,66.46,8/8/2020 2:45,TPH.RH,%,144.966498,-37.800780,8
1048572,1542107,1004,arc1048,0b,17.60,8/8/2020 3:45,PM10,ug/m^3,144.964641,-37.800562,8
1048573,1542108,1003,arc1047,6,4.18,8/8/2020 3:15,WS,km/h,144.960929,-37.802287,8


In [98]:
df_temp = df[df['units']=="C"]
df_temp = df_temp[['month', 'value', 'site_id']].groupby(['month']).mean('value')
df_temp.tail(60)

,value,site_id
month,,
1,20.719237,1003.108706
2,19.694046,1003.047492
3,18.377807,1002.997394
4,15.362876,1003.005765
5,12.095714,1003.272373
6,11.088363,1003.068940
7,10.388505,1003.066004
8,9.685775,1003.082295
9,15.187273,1012.363636


In [70]:
df_temp = df[df['units']=="C"]
df_temp = df_temp[['month', 'value']].groupby(['month']).mean('value')
df_temp = df_temp.add_suffix('_mean').reset_index()

# Round values
df_temp['value_mean'] = round(df_temp['value_mean'],3)

df_temp.tail(30)

,month,value_mean
0,1,20.719
1,2,19.694
2,3,18.378
3,4,15.363
4,5,12.096
5,6,11.088
6,7,10.389
7,8,9.686
8,9,15.187
9,11,17.221


In [79]:
# Read in data and create dataframe
path = home + 'data/Environment/Microclimate_Sensor_Readings.csv'
df_temp = pd.read_csv(path)

# Filter for temperature and group by lat long
df_temp = df_temp[df_temp['units']=="C"]
df_temp = df_temp[['site_id', 'longitude', 'latitude', 'value']].groupby(['site_id', 'latitude', 'longitude']).mean()
df_temp = df_temp.add_suffix('_mean').reset_index()

# Round values
df_temp['value_mean'] = round(df_temp['value_mean'],2) 

In [80]:
df_temp

,site_id,latitude,longitude,value_mean
0,1001,-37.800780,144.966498,16.84
1,1002,-37.800511,144.964128,15.88
2,1003,-37.802287,144.960929,16.01
3,1004,-37.800562,144.964641,15.68
4,1005,-37.800616,144.965058,15.26
5,1007,-37.822447,144.951841,13.71
6,1009,-37.816854,144.965711,11.22
7,1010,-37.822487,144.952228,10.81
8,1011,-37.822209,144.952228,11.34
9,1012,-37.813320,144.970006,12.76


In [81]:
# Wind Speed
path = home + 'data/Environment/Microclimate_Sensor_Readings.csv'
df_ws = pd.read_csv(path)

# Filter for windspeed
df_ws = df_ws[df_ws['units']=="km/h"]
df_ws = df_ws[['site_id', 'longitude', 'latitude', 'value']].groupby(['site_id', 'latitude', 'longitude']).mean()
df_ws = df_ws.add_suffix('_mean').reset_index()

# Round values
df_ws['value_mean'] = round(df_ws['value_mean'],2) 

In [82]:
df_ws

,site_id,latitude,longitude,value_mean
0,1001,-37.800780,144.966498,3.42
1,1002,-37.800511,144.964128,7.31
2,1003,-37.802287,144.960929,4.45
3,1004,-37.800562,144.964641,6.77
4,1005,-37.800616,144.965058,6.11
5,1007,-37.822447,144.951841,4.10
6,1009,-37.816854,144.965711,4.57
7,1010,-37.822487,144.952228,5.34
8,1011,-37.822209,144.952228,6.75
9,1012,-37.813320,144.970006,9.48


In [83]:
# Wind Speed
path = home + 'data/Environment/Microclimate_Sensor_Readings.csv'
df_hum = pd.read_csv(path)

# Filter for windspeed
df_hum = df_hum[df_hum['units']=="%"]
df_hum = df_hum[['site_id', 'longitude', 'latitude', 'value']].groupby(['site_id', 'latitude', 'longitude']).mean()
df_hum = df_hum.add_suffix('_mean').reset_index()

# Round values
df_hum['value_mean'] = round(df_hum['value_mean'],2) 

In [84]:
df_hum

,site_id,latitude,longitude,value_mean
0,1001,-37.800780,144.966498,59.59
1,1002,-37.800511,144.964128,67.08
2,1003,-37.802287,144.960929,68.23
3,1004,-37.800562,144.964641,66.85
4,1005,-37.800616,144.965058,66.82
5,1007,-37.822447,144.951841,66.79
6,1009,-37.816854,144.965711,67.43
7,1010,-37.822487,144.952228,68.24
8,1011,-37.822209,144.952228,74.59
9,1012,-37.813320,144.970006,66.39


In [85]:
df_means = df_temp.copy()
df_means.rename(columns={'value_mean':'temp_mean'}, inplace=True)
df_means['humidity_mean'] = df_hum['value_mean'].tolist()
df_means['ws_mean'] = df_ws['value_mean'].tolist()

df_means

,site_id,latitude,longitude,temp_mean,humidity_mean,ws_mean
0,1001,-37.800780,144.966498,16.84,59.59,3.42
1,1002,-37.800511,144.964128,15.88,67.08,7.31
2,1003,-37.802287,144.960929,16.01,68.23,4.45
3,1004,-37.800562,144.964641,15.68,66.85,6.77
4,1005,-37.800616,144.965058,15.26,66.82,6.11
5,1007,-37.822447,144.951841,13.71,66.79,4.10
6,1009,-37.816854,144.965711,11.22,67.43,4.57
7,1010,-37.822487,144.952228,10.81,68.24,5.34
8,1011,-37.822209,144.952228,11.34,74.59,6.75
9,1012,-37.813320,144.970006,12.76,66.39,9.48


In [86]:
# Save to csv

path = home + 'data/Environment/Microclimate_Sensor_Readings_mean.csv'
df_means.to_csv(path)

## Coworking spaces

In [25]:
path = home + 'data/Places_Of_Interest/Coworking_Spaces.csv'
df = pd.read_csv(path)
df

,Organisation,Address,Website,latitude,longitude,location
0,@workspaces,"Level 5, 171 Collins Street, Melbourne 3000",https://www.atworkspaces.com/locations/melbourne,-37.815447,144.968789,POINT (144.968789 -37.815447)
1,11th Space,"Level 11/580 Collins St, Melbourne VIC 3000",https://11thspace.com/,-37.818523,144.955364,POINT (144.9553637 -37.818523)
2,25 King Collective,"25 King Street, Melbourne 3000",https://www.25king.com.au/,-37.819840,144.957030,POINT (144.95703 -37.81984)
3,360 Collins,"360 Collins Street, Melbourne 3000",https://officespace.com.au/melbourne/153369/,-37.816002,144.962311,POINT (144.962311 -37.816002)
4,ACMI X,"Level 4, 2 Kavanagh Street, Southbank 3006",https://www.acmi.net.au/acmi-x/,-37.821801,144.967594,POINT (144.9675938 -37.8218014)
...,...,...,...,...,...,...
61,Workspace365,"Ground Floor, 555 Bourke Street Melbourne VIC ...",https://www.workspace365.com.au/co-working,-37.816202,144.957406,POINT (144.957406 -37.816202)
62,Workspace365,"Ground Floor, 485 La Trobe Street, Melbourne V...",https://www.workspace365.com.au/co-working,-37.812563,144.954838,POINT (144.954838 -37.812563)
63,Workspace365,"607 Bourke Street, Melbourne 3000",https://www.workspace365.com.au/co-working,-37.816673,144.955505,POINT (144.955505 -37.816673)
64,Workspace365,"Level 9/14/15 330 Collins Street, Melbourne 3000",https://www.workspace365.com.au/co-working,-37.816211,144.963253,POINT (144.9632531 -37.816211)


In [24]:
locations = df['location'].tolist()
pts = [shapely.wkt.loads(loc) for loc in locations]
longs = [pt.x for pt in pts]
lats = [pt.y for pt in pts]

df['longitude'] = longs
df['latitude'] = lats


## Parking bays

In [5]:
import fiona

shape = fiona.open("/Users/declanwatson/Desktop/On-street Parking Bays/geo_export_f9c2264a-cd33-4c8c-a871-d985ff726acc.shp")

print(shape.schema)
first = shape.next()

from shapely.geometry import shape

shp_geom = shape(first['geometry'])

print(shp_geom.centroid)


{'properties': OrderedDict([('bay_id', 'str:254'), ('last_edit', 'str:254'), ('marker_id', 'str:254'), ('meter_id', 'str:254'), ('rd_seg_dsc', 'str:254'), ('rd_seg_id', 'str:254')]), 'geometry': 'Polygon'}


/var/folders/0q/cgrxp74x1l7bxnzhxcgnpmf00000gn/T/ipykernel_48626/1472574359.py:6: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  first = shape.next()


In [15]:
import geopandas as gpd

shapefile = gpd.read_file("/Users/declanwatson/Desktop/On-street Parking Bays/geo_export_f9c2264a-cd33-4c8c-a871-d985ff726acc.shp")
shapefile

shps = [shape(geo) for geo in shapefile['geometry'].tolist()]

In [21]:
# Extract long and lats
longs = [pt.centroid.x for pt in shps]
lats = [pt.centroid.y for pt in shps]

In [22]:
# save to dataframe

df = pd.DataFrame()
df['longitude'] = longs
df['latitude'] = lats

In [24]:
df.to_csv("/Users/declanwatson/Desktop/street_parking.csv" , index=False)